# Segragate your tenants by namespace and by nodes

### Prerequisites
1. An AKS cluster (you can create them on the console or via the CLI)
2. az cli installed and logged in
3. A storage account with two folders. Example: \
    ngjasonmultistorage (storage account) \
        - tenanta (container) \
        - tenantb (container)
4. Upload random files to tenanta and tenantb container
5. You have a bash kernel if you are running on windows wsl
6. AI foundry project with gpt-4.1 and gpt-4.1-mini (or any model that you want, just substitute within main.py)

### What you will create
1. Create nodes with taints and labels for each specific tenant
2. Create tolerations and node selectors on pod manifest so that they get deployed on the respective tenant node pool (app isolation)
3. Create managed identities, the pods will use the identity via WorkloadIdentityCredential to connect to blob storage
4. Create service accounts to manage federated credentials
5. Create RBAC and ABAC and assigned them to respective managed identities (storage isolation)

![image.png](image.png)

## Connect to your cluster and export variables. Replace with your own values

In [1]:
%env cluster_name=ngjason-akscluster
%env resource_group=aks-gpu
%env storage_account_name=ngjasonmultistorage
%env tenant_a_container=tenanta
%env tenant_b_container=tenantb
%env subscription_id=bb608350-d9c5-4b2a-b61e-6df4034dbf07

env: cluster_name=ngjason-akscluster
env: resource_group=aks-gpu
env: storage_account_name=ngjasonmultistorage
env: tenant_a_container=tenanta
env: tenant_b_container=tenantb
env: subscription_id=bb608350-d9c5-4b2a-b61e-6df4034dbf07


In [2]:
%%bash
echo $cluster_name
echo $resource_group
echo $storage_account_name
echo $tenant_a_container
echo $tenant_b_container
echo $subscription_id

ngjason-akscluster
aks-gpu
ngjasonmultistorage
tenanta
tenantb
bb608350-d9c5-4b2a-b61e-6df4034dbf07


## Connect to your cluster - Open a terminal where this jupyter notebook is running and run the command so that kubectl persists across all cells.

<pre>az aks get-credentials --name $cluster_name -g $resource_group</pre>

## Enable workload identity on the cluster (This is a one-time setup and can take a while to complete, skip if you have already enabled this before)

In [ ]:
%%bash
az aks update --resource-group $resource_group --name $cluster_name \
  --enable-oidc-issuer \
  --enable-workload-identity


## Create namespaces for each tenant

In [3]:
%%bash
kubectl create namespace tenant-a
kubectl create namespace tenant-b

namespace/tenant-a created
namespace/tenant-b created


## Create two node pools, one for each tenant

In [24]:
%%bash
az aks nodepool add \
  --name tenantpoola \
  --cluster-name $cluster_name \
  --resource-group $resource_group \
  --node-taints tenantid=tenanta:NoSchedule \
  --labels tenantid=tenanta \
  --node-vm-size Standard_D4s_v3

az aks nodepool add \
  --name tenantpoolb \
  --cluster-name $cluster_name \
  --resource-group $resource_group \
  --node-taints tenantid=tenantb:NoSchedule \
  --labels tenantid=tenantb \ 
  --node-vm-size Standard_D4s_v3

/home/ngjason/.azure/cliextensions/aks-preview/azext_aks_preview/vendored_sdks/__init__.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__('pkg_resources').declare_namespace(__name__)


{
  "artifactStreamingProfile": null,
  "availabilityZones": null,
  "capacityReservationGroupId": null,
  "count": 3,
  "creationData": null,
  "currentOrchestratorVersion": "1.32.5",
  "eTag": "c4ab5dee-201c-4910-8bd4-6c834ee11c16",
  "enableAutoScaling": false,
  "enableCustomCaTrust": false,
  "enableEncryptionAtHost": false,
  "enableFips": false,
  "enableNodePublicIp": false,
  "enableUltraSsd": false,
  "gatewayProfile": null,
  "gpuInstanceProfile": null,
  "gpuProfile": null,
  "hostGroupId": null,
  "id": "/subscriptions/bb608350-d9c5-4b2a-b61e-6df4034dbf07/resourcegroups/aks-gpu/providers/Microsoft.ContainerService/managedClusters/ngjason-akscluster/agentPools/tenantpoola",
  "kubeletConfig": null,
  "kubeletDiskType": "OS",
  "linuxOsConfig": null,
  "localDnsProfile": null,
  "maxCount": null,
  "maxPods": 250,
  "messageOfTheDay": null,
  "minCount": null,
  "mode": "User",
  "name": "tenantpoola",
  "networkProfile": {
    "allowedHostPorts": null,
    "applicationSecur

/home/ngjason/.azure/cliextensions/aks-preview/azext_aks_preview/vendored_sdks/__init__.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__('pkg_resources').declare_namespace(__name__)
ERROR: Invalid label:  . Label definition must be of format name=value.
bash: line 15: --node-vm-size: command not found


CalledProcessError: Command 'b'az aks nodepool add \\\n  --name tenantpoola \\\n  --cluster-name $cluster_name \\\n  --resource-group $resource_group \\\n  --node-taints tenantid=tenanta:NoSchedule \\\n  --labels tenantid=tenanta \\\n  --node-vm-size Standard_D4s_v3\n\naz aks nodepool add \\\n  --name tenantpoolb \\\n  --cluster-name $cluster_name \\\n  --resource-group $resource_group \\\n  --node-taints tenantid=tenantb:NoSchedule \\\n  --labels tenantid=tenantb \\ \n  --node-vm-size Standard_D4s_v3\n'' returned non-zero exit status 127.

## Create managed identities for each tenant

In [5]:
%%bash
az identity create --resource-group aks-gpu --name mi-tenant-a
az identity create --resource-group aks-gpu --name mi-tenant-b

{
  "clientId": "7d4712d9-20c4-45e8-b974-7d3e3baf7ca1",
  "id": "/subscriptions/bb608350-d9c5-4b2a-b61e-6df4034dbf07/resourcegroups/aks-gpu/providers/Microsoft.ManagedIdentity/userAssignedIdentities/mi-tenant-a",
  "location": "eastus2",
  "name": "mi-tenant-a",
  "principalId": "9a9f75ee-0b4a-4f90-a933-81e927f69af5",
  "resourceGroup": "aks-gpu",
  "systemData": null,
  "tags": {},
  "tenantId": "16b3c013-d300-468d-ac64-7eda0820b6d3",
  "type": "Microsoft.ManagedIdentity/userAssignedIdentities"
}
{
  "clientId": "50032f0b-9a7c-41e6-ac39-f3181b6c7fb0",
  "id": "/subscriptions/bb608350-d9c5-4b2a-b61e-6df4034dbf07/resourcegroups/aks-gpu/providers/Microsoft.ManagedIdentity/userAssignedIdentities/mi-tenant-b",
  "location": "eastus2",
  "name": "mi-tenant-b",
  "principalId": "8e71fbf8-4d3e-40e3-b0d0-2af97a58a0f1",
  "resourceGroup": "aks-gpu",
  "systemData": null,
  "tags": {},
  "tenantId": "16b3c013-d300-468d-ac64-7eda0820b6d3",
  "type": "Microsoft.ManagedIdentity/userAssignedIdentiti

## Create RBAC and ABAC policies for each container within blob storage allowing managed identity to connect

In [18]:
%%bash
az role assignment create \
  --assignee $(az identity show --name mi-tenant-a --resource-group aks-gpu --query principalId -o tsv) \
  --role "Storage Blob Data Contributor" \
  --scope "/subscriptions/${subscription_id}/resourceGroups/${resource_group}/providers/Microsoft.Storage/storageAccounts/${storage_account_name}/blobServices/default/containers/tenanta" \
  --condition "((!(ActionMatches{'Microsoft.Storage/storageAccounts/blobServices/containers/blobs/*'})) OR (@Resource[Microsoft.Storage/storageAccounts/blobServices/containers:name] StringEquals 'tenanta'))" \
  --condition-version "2.0"
  
az role assignment create \
  --assignee $(az identity show --name mi-tenant-b --resource-group aks-gpu --query principalId -o tsv) \
  --role "Storage Blob Data Contributor" \
  --scope "/subscriptions/${subscription_id}/resourceGroups/${resource_group}/providers/Microsoft.Storage/storageAccounts/${storage_account_name}/blobServices/default/containers/tenantb" \
  --condition "((!(ActionMatches{'Microsoft.Storage/storageAccounts/blobServices/containers/blobs/*'})) OR (@Resource[Microsoft.Storage/storageAccounts/blobServices/containers:name] StringEquals 'tenantb'))" \
  --condition-version "2.0"

{
  "condition": "((!(ActionMatches{'Microsoft.Storage/storageAccounts/blobServices/containers/blobs/*'})) OR (@Resource[Microsoft.Storage/storageAccounts/blobServices/containers:name] StringEquals 'tenanta'))",
  "conditionVersion": "2.0",
  "createdBy": "3ce00cd3-fa53-4787-bee8-577cc1b1f68f",
  "createdOn": "2025-07-01T09:51:56.960773+00:00",
  "delegatedManagedIdentityResourceId": null,
  "description": null,
  "id": "/subscriptions/bb608350-d9c5-4b2a-b61e-6df4034dbf07/resourceGroups/aks-gpu/providers/Microsoft.Storage/storageAccounts/ngjasonmultistorage/blobServices/default/containers/tenanta/providers/Microsoft.Authorization/roleAssignments/b642e93c-2c0a-45b9-8cd7-d3e3c3d35b3d",
  "name": "b642e93c-2c0a-45b9-8cd7-d3e3c3d35b3d",
  "principalId": "9a9f75ee-0b4a-4f90-a933-81e927f69af5",
  "principalName": "7d4712d9-20c4-45e8-b974-7d3e3baf7ca1",
  "principalType": "ServicePrincipal",
  "resourceGroup": "aks-gpu",
  "roleDefinitionId": "/subscriptions/bb608350-d9c5-4b2a-b61e-6df4034dbf

{
  "condition": "((!(ActionMatches{'Microsoft.Storage/storageAccounts/blobServices/containers/blobs/*'})) OR (@Resource[Microsoft.Storage/storageAccounts/blobServices/containers:name] StringEquals 'tenantb'))",
  "conditionVersion": "2.0",
  "createdBy": "3ce00cd3-fa53-4787-bee8-577cc1b1f68f",
  "createdOn": "2025-07-01T09:52:08.662860+00:00",
  "delegatedManagedIdentityResourceId": null,
  "description": null,
  "id": "/subscriptions/bb608350-d9c5-4b2a-b61e-6df4034dbf07/resourceGroups/aks-gpu/providers/Microsoft.Storage/storageAccounts/ngjasonmultistorage/blobServices/default/containers/tenantb/providers/Microsoft.Authorization/roleAssignments/33f84ddc-de17-4f82-a9df-e33dcf956edf",
  "name": "33f84ddc-de17-4f82-a9df-e33dcf956edf",
  "principalId": "8e71fbf8-4d3e-40e3-b0d0-2af97a58a0f1",
  "principalName": "50032f0b-9a7c-41e6-ac39-f3181b6c7fb0",
  "principalType": "ServicePrincipal",
  "resourceGroup": "aks-gpu",
  "roleDefinitionId": "/subscriptions/bb608350-d9c5-4b2a-b61e-6df4034dbf

## Create Service account and federated credential. Pods will use service account to obtain temporary credentials via OIDC

In [7]:
%%bash
export tenant_a_identity_id=$(az identity show --name mi-tenant-a --resource-group $resource_group --query clientId -o tsv)
export tenant_b_identity_id=$(az identity show --name mi-tenant-b --resource-group $resource_group --query clientId -o tsv)

kubectl apply -n tenant-a -f - <<EOF
apiVersion: v1
kind: ServiceAccount
metadata:
  name: storage-access-sa-tenant-a
  namespace: tenant-a
  annotations:
    azure.workload.identity/client-id: $tenant_a_identity_id
  labels:
    azure.workload.identity/use: "true"
EOF

kubectl apply -n tenant-b -f - <<EOF
apiVersion: v1
kind: ServiceAccount
metadata:
  name: storage-access-sa-tenant-b
  namespace: tenant-b
  annotations:
    azure.workload.identity/client-id: $tenant_b_identity_id
  labels:
    azure.workload.identity/use: "true"
EOF

serviceaccount/storage-access-sa-tenant-a created
serviceaccount/storage-access-sa-tenant-b created


In [8]:
%%bash
export SERVICE_ACCOUNT_ISSUER=$(az aks show \
  --resource-group $resource_group \
  --name $cluster_name \
  --query oidcIssuerProfile.issuerUrl -o tsv)

echo $SERVICE_ACCOUNT_ISSUER

az identity federated-credential create \
  --name "aks-federated-credential" \
  --identity-name mi-tenant-a \
  --resource-group $resource_group \
  --issuer $SERVICE_ACCOUNT_ISSUER \
  --subject "system:serviceaccount:tenant-a:storage-access-sa-tenant-a"

az identity federated-credential create \
  --name "aks-federated-credential" \
  --identity-name mi-tenant-b \
  --resource-group $resource_group \
  --issuer $SERVICE_ACCOUNT_ISSUER \
  --subject "system:serviceaccount:tenant-b:storage-access-sa-tenant-b"

/home/ngjason/.azure/cliextensions/aks-preview/azext_aks_preview/vendored_sdks/__init__.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__('pkg_resources').declare_namespace(__name__)


https://eastus2.oic.prod-aks.azure.com/16b3c013-d300-468d-ac64-7eda0820b6d3/a9002abf-667e-4f02-a436-d9bd423afe81/
{
  "audiences": [
    "api://AzureADTokenExchange"
  ],
  "id": "/subscriptions/bb608350-d9c5-4b2a-b61e-6df4034dbf07/resourcegroups/aks-gpu/providers/Microsoft.ManagedIdentity/userAssignedIdentities/mi-tenant-a/federatedIdentityCredentials/aks-federated-credential",
  "issuer": "https://eastus2.oic.prod-aks.azure.com/16b3c013-d300-468d-ac64-7eda0820b6d3/a9002abf-667e-4f02-a436-d9bd423afe81/",
  "name": "aks-federated-credential",
  "resourceGroup": "aks-gpu",
  "subject": "system:serviceaccount:tenant-a:storage-access-sa-tenant-a",
  "type": "Microsoft.ManagedIdentity/userAssignedIdentities/federatedIdentityCredentials"
}
{
  "audiences": [
    "api://AzureADTokenExchange"
  ],
  "id": "/subscriptions/bb608350-d9c5-4b2a-b61e-6df4034dbf07/resourcegroups/aks-gpu/providers/Microsoft.ManagedIdentity/userAssignedIdentities/mi-tenant-b/federatedIdentityCredentials/aks-federated-

## Create test pods

## Check that pods are scheduled on the correct node

In [9]:
%%bash

kubectl apply -n tenant-a -f - <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: tenant-a-app
  labels:
    azure.workload.identity/use: "true"  # Required for workload identity
spec:
  selector:
    matchLabels:
      tenantid: tenanta
  template:
    metadata:
      labels:
        tenantid: tenanta
        azure.workload.identity/use: "true"  # Must be on pod template
    spec:
      serviceAccountName: storage-access-sa-tenant-a  # Reference your service account
      nodeSelector:
        tenantid: tenanta
      tolerations:
      - key: "tenantid"
        operator: "Equal"
        value: "tenanta"
        effect: "NoSchedule"
      containers:
      - name: nginx
        image: nginx
        env:
          - name: TENANT_ID
            value: "tenanta"  # Environment variable to identify tenant
EOF

kubectl apply -n tenant-b -f - <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: tenant-b-app
  labels:
    azure.workload.identity/use: "true"  # Required for workload identity
spec:
  selector:
    matchLabels:
      tenantid: tenantb
  template:
    metadata:
      labels:
        tenantid: tenantb
        azure.workload.identity/use: "true"  # Must be on pod template
    spec:
      serviceAccountName: storage-access-sa-tenant-b  # Reference your service account
      nodeSelector:
        tenantid: tenantb
      tolerations:
      - key: "tenantid"
        operator: "Equal"
        value: "tenantb"
        effect: "NoSchedule"
      containers:
      - name: nginx
        image: nginx
        env:
          - name: TENANT_ID
            value: "tenantb"
EOF

deployment.apps/tenant-a-app created
deployment.apps/tenant-b-app created


### If you see your pods stuck in pending state, it is most likely that you dont have the correct labels on the nodes. Check from the portal if the taints and labels are set correctly

In [10]:
%%bash
kubectl get pods -o wide -n tenant-a
kubectl get pods -o wide -n tenant-b

NAME                            READY   STATUS    RESTARTS   AGE   IP             NODE                                  NOMINATED NODE   READINESS GATES
tenant-a-app-5bfbbd8477-9fjk4   1/1     Running   0          5s    10.244.4.200   aks-tenantpoola-12478992-vmss000002   <none>           <none>
NAME                          READY   STATUS    RESTARTS   AGE   IP            NODE                                  NOMINATED NODE   READINESS GATES
tenant-b-app-c798fbf5-srsxv   1/1     Running   0          5s    10.244.7.65   aks-tenantpoolb-15486632-vmss000000   <none>           <none>


## Test out the tenant isolation by making calls to blob storage

**Update the blob account url with your own blob storage url in app.py**

Expected results:\
pod A -> tenant A container ✅ \
pod A -> tenant B container ❌ \
\
pod B -> tenant A container ❌ \
pod B -> tenant B container ✅


In [ ]:
%%bash
export tenant_a_pod=$(kubectl get pods -n tenant-a -o jsonpath='{.items[0].metadata.name}')
kubectl cp -n tenant-a  ./app.py $tenant_a_pod:/tmp/app.py
kubectl cp -n tenant-a  ./install.sh $tenant_a_pod:/tmp/install.sh
kubectl exec -it pod/$tenant_a_pod -n tenant-a -- /bin/bash -c "chmod +x /tmp/install.sh && /tmp/install.sh"

In [ ]:
%%bash
export tenant_b_pod=$(kubectl get pods -n tenant-b -o jsonpath='{.items[0].metadata.name}')
kubectl cp -n tenant-b  ./app.py $tenant_b_pod:/tmp/app.py
kubectl cp -n tenant-b  ./install.sh $tenant_b_pod:/tmp/install.sh
kubectl exec -it pod/$tenant_b_pod -n tenant-b -- /bin/bash -c "chmod +x /tmp/install.sh && /tmp/install.sh"

In [22]:
%%bash
export tenant_a_pod=$(kubectl get pods -n tenant-a -o jsonpath='{.items[0].metadata.name}')
kubectl exec -it pod/$tenant_a_pod -n tenant-a -- /bin/bash -c "python3 /tmp/app.py"

Unable to use a TTY - input is not a terminal or the right kind of file


Connecting to Azure Blob Storage for Tenant A
tenant-a-file.txt
✅ Successfully connected to tenant A container
Connecting to Azure Blob Storage for Tenant B
An error occurred: This request is not authorized to perform this operation using this permission.
RequestId:d7e7297f-e01e-00e3-1ae4-fa074a000000
Time:2025-07-22T08:42:16.2270832Z
ErrorCode:AuthorizationPermissionMismatch
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>AuthorizationPermissionMismatch</Code><Message>This request is not authorized to perform this operation using this permission.
RequestId:d7e7297f-e01e-00e3-1ae4-fa074a000000
Time:2025-07-22T08:42:16.2270832Z</Message></Error>
❌ Failed to connect to tenant B container


In [21]:
%%bash
export tenant_b_pod=$(kubectl get pods -n tenant-b -o jsonpath='{.items[0].metadata.name}')
kubectl exec -it pod/$tenant_b_pod -n tenant-b -- /bin/bash -c "python3 /tmp/app.py"

Unable to use a TTY - input is not a terminal or the right kind of file


Connecting to Azure Blob Storage for Tenant A
An error occurred: This request is not authorized to perform this operation using this permission.
RequestId:eccf259b-a01e-0074-44e4-fa5147000000
Time:2025-07-22T08:42:06.9807573Z
ErrorCode:AuthorizationPermissionMismatch
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>AuthorizationPermissionMismatch</Code><Message>This request is not authorized to perform this operation using this permission.
RequestId:eccf259b-a01e-0074-44e4-fa5147000000
Time:2025-07-22T08:42:06.9807573Z</Message></Error>
❌ Failed to connect to tenant A container
Connecting to Azure Blob Storage for Tenant B
tenant-b-file.txt
✅ Successfully connected to tenant B container
